In [1]:
# This script demonstrates the capabilities of cluster_slug by using
# it to re-analyze the M31 star clusters catalog of Johnson et al. (2015, ApJ)

# Import the libraries we'll need
from astropy.io import ascii
from astropy import table
from astropy.table import unique, Table, hstack, Column
import sys
from numpy import inf
import numpy as np
import numpy.ma as ma
import time
from matplotlib.pyplot import figure as fig
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from slugpy.cluster_slug import cluster_slug
from slugpy import * 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [16]:
#Imported AP (M33) fits files which include best fit of mass and age data already

m33phot_test=Table.read('/Users/user/Dropbox/m33_zoo_data/data_v2/apm33_6phot_test.fits')

LEGUS_PHAT_PadKroupa_prop=Table.read('LEGUS_1e6_1e10age_PadKroupaAfter_PHAT_lib_1e7Trials_cluster_prop.fits')
LEGUS_PHAT_PadKroupa_phot=Table.read('LEGUS_1e6_1e10age_PadKroupaAfter_PHAT_lib_1e7Trials_cluster_phot.fits')

dmod_M33=24.65 #De Grijs et al 2014 (AJ 148:17)

In [17]:
m33phot_test.remove_columns(['MAG110', 'SIG110', 'DETSIG110'])

In [18]:
m33phot_test.remove_columns(['MAG160', 'SIG160', 'DETSIG160'])

In [19]:
ind275DS3=np.where(m33phot_test['DETSIG275']<3.0)
m33phot_test.remove_rows(ind275DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG275,SIG275,DETSIG275,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.731682,0.15761206,4.439818,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.336721,0.11588563,6.022165,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,18.033772,0.090995274,7.6567597,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.234766,0.016391266,42.46132,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.788614,0.17869133,3.9248168,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.817127,0.109852366,6.3534365,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.084024,0.028913893,24.063679,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,122,10709,M33-B02-F09_21,2,9,21,-1,28525031,2,0,"81.7,142.8,","0.9833,0.9355,","0,0,",12.033003,17.86079,0.045716606,15.2228775,17.892584,0.03414037,20.372658,18.181175,0.048195247,22.543873,17.363104,0.124078065,8.78851
25,188.74258,416.76825,43.118168,1438.7426,1616.7683,23.506394732479563,30.70860583353485,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,8204,M33-B02-F03_15,2,3,15,-1,28524857,2,0,"83.2,17.4,","0.9833,0.9333,","0,1,",10.272879,17.606758,0.050595663,13.751336,17.660112,0.017234653,40.389404,17.993713,0.026560588,40.89101,17.556812,0.12135625,8.983954


In [20]:
ind336DS3=np.where(m33phot_test['DETSIG336']<3.0)
m33phot_test.remove_rows(ind336DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG275,SIG275,DETSIG275,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.731682,0.15761206,4.439818,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.336721,0.11588563,6.022165,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,18.033772,0.090995274,7.6567597,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.234766,0.016391266,42.46132,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.788614,0.17869133,3.9248168,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.817127,0.109852366,6.3534365,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.084024,0.028913893,24.063679,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,122,10709,M33-B02-F09_21,2,9,21,-1,28525031,2,0,"81.7,142.8,","0.9833,0.9355,","0,0,",12.033003,17.86079,0.045716606,15.2228775,17.892584,0.03414037,20.372658,18.181175,0.048195247,22.543873,17.363104,0.124078065,8.78851
25,188.74258,416.76825,43.118168,1438.7426,1616.7683,23.506394732479563,30.70860583353485,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,8204,M33-B02-F03_15,2,3,15,-1,28524857,2,0,"83.2,17.4,","0.9833,0.9333,","0,1,",10.272879,17.606758,0.050595663,13.751336,17.660112,0.017234653,40.389404,17.993713,0.026560588,40.89101,17.556812,0.12135625,8.983954


In [21]:
ind475DS3=np.where(m33phot_test['DETSIG475']<3.0)
m33phot_test.remove_rows(ind475DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG275,SIG275,DETSIG275,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.731682,0.15761206,4.439818,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.336721,0.11588563,6.022165,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,18.033772,0.090995274,7.6567597,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.234766,0.016391266,42.46132,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.788614,0.17869133,3.9248168,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.817127,0.109852366,6.3534365,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.084024,0.028913893,24.063679,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,122,10709,M33-B02-F09_21,2,9,21,-1,28525031,2,0,"81.7,142.8,","0.9833,0.9355,","0,0,",12.033003,17.86079,0.045716606,15.2228775,17.892584,0.03414037,20.372658,18.181175,0.048195247,22.543873,17.363104,0.124078065,8.78851
25,188.74258,416.76825,43.118168,1438.7426,1616.7683,23.506394732479563,30.70860583353485,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,8204,M33-B02-F03_15,2,3,15,-1,28524857,2,0,"83.2,17.4,","0.9833,0.9333,","0,1,",10.272879,17.606758,0.050595663,13.751336,17.660112,0.017234653,40.389404,17.993713,0.026560588,40.89101,17.556812,0.12135625,8.983954


In [22]:
ind814DS3=np.where(m33phot_test['DETSIG814']<3.0)
m33phot_test.remove_rows(ind814DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG275,SIG275,DETSIG275,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.731682,0.15761206,4.439818,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917
11,119.026726,403.0801,33.970577,744.02673,2403.08,23.445514019227883,30.7663251695561,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,7146,M33-B01-F18_22,1,18,22,-1,28524780,2,0,"96.9,8.8,","0.9833,0.7167,","0,1,",10.902826,19.336721,0.11588563,6.022165,19.508385,0.14812975,4.720739,19.481394,0.05234097,20.761887,19.410734,0.36016145,3.124356
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,18.033772,0.090995274,7.6567597,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444
15,611.2133,299.76617,52.04386,2486.2134,299.76617,23.577712900686066,30.52706255699551,2,0.98333335,0.9830508,0.016949153,0.0,0.016949153,116,0,0,0,2,0,120,26228,M33-B03-F07-TE_04,3,7,4,1,28526102,3,0,"113.8,56.8,20.4,","0.9833,0.9833,0.9000,","0,0,1,",11.958765,16.234766,0.016391266,42.46132,16.356699,0.01643354,42.310066,16.84331,0.02626825,41.34225,16.140528,0.0340229,31.922668
16,438.82727,246.76823,71.65176,2313.8271,1446.7682,23.605610102605468,30.691192015762407,1,0.98333335,0.8305085,0.16949153,0.06779661,0.10169491,49,0,4,0,6,0,60,27000,M33-B02-F01-TE_16,2,1,16,1,28526170,1,0,246.8,0.9833,0,20.366236,18.788614,0.17869133,3.9248168,18.07968,0.10024265,6.952229,17.810106,0.0048297485,225.50786,16.615978,0.06446845,16.861229
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.817127,0.109852366,6.3534365,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.084024,0.028913893,24.063679,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,122,10709,M33-B02-F09_21,2,9,21,-1,28525031,2,0,"81.7,142.8,","0.9833,0.9355,","0,0,",12.033003,17.86079,0.045716606,15.2228775,17.892584,0.03414037,20.372658,18.181175,0.048195247,22.543873,17.363104,0.124078065,8.78851
25,188.74258,416.76825,43.118168,1438.7426,1616.7683,23.506394732479563,30.70860583353485,1,0.98333335,1.0,0.0,0.0,0.0,59,0,0,0,0,0,60,8204,M33-B02-F03_15,2,3,15,-1,28524857,2,0,"83.2,17.4,","0.9833,0.9333,","0,1,",10.272879,17.606758,0.050595663,13.751336,17.660112,0.017234653,40.389404,17.993713,0.026560588,40.89101,17.556812,0.12135625,8.983954


In [15]:
ind110DS3=np.where(m33phot_test['DETSIG110']<3.0)
m33phot_test.remove_rows(ind110DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814,MAG110,SIG110,DETSIG110
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
4,274.12375,430.10718,38.054737,1524.1238,430.10718,23.43433303773043,30.51408232617703,1,1.0,0.98333335,0.016666668,0.0,0.016666668,59,0,0,0,1,0,60,21254,M33-B03-F16_03,3,16,3,-1,28525713,2,0,"69.9,28.9,","1.0000,0.9000,","0,1,",13.384977,17.207281,0.07225124,9.632259,19.698397,0.08457241,12.864533,18.16659,0.15653984,6.9838634,17.306377,0.16535696,6.616979
5,224.73451,423.41208,37.18926,849.7345,1623.4121,23.583845918691235,30.659212182808222,1,0.9836066,0.9166667,0.083333336,0.0,0.083333336,55,0,0,0,5,0,61,9815,M33-B02-F07_14,2,7,14,-1,28524968,2,0,"76.6,23.6,","0.9836,0.8548,","0,1,",9.4648075,18.9727,0.11130678,6.2656794,18.70199,0.023569176,46.085743,17.94467,0.06835622,15.904741,17.577536,0.12523706,8.708436
7,248.0658,140.99915,39.348427,873.0658,140.99915,23.390373394289195,30.6674512161078,2,0.96694213,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,121,11623,M33-B02-F12_02,2,12,2,-1,28525096,2,0,"83.6,141.0,","0.9833,0.9508,","0,0,",12.779946,19.286543,0.14845498,4.710332,18.826597,0.08265972,13.160799,17.740986,0.16589268,6.5956807,17.166212,0.20642294,5.3231254
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917,15.6878,0.10127982,10.751449
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444,16.555428,0.10290856,10.582573
17,279.86597,185.6878,50.159744,1529.866,2585.6877,23.52419243700545,30.83246332535244,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,1496,M33-B01-F04_27,1,4,27,-1,28524393,1,0,185.7,0.9833,0,12.008762,17.310535,0.0974104,7.152913,17.800167,0.0681135,15.961256,17.484009,0.08811067,12.349536,17.155066,0.13650873,7.995891
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064,18.891033,0.18725759,5.856528
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059,16.69304,0.109354645,9.962718
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,122,10709,M33-B02-F09_21,2,9,21,-1,28525031,2,0,"81.7,142.8,","0.9833,0.9355,","0,0,",12.033003,17.892584,0.03414037,20.372658,18.181175,0.048195247,22.543873,17.363104,0.124078065,8.78851,16.862604,0.1363013,8.007986


In [8]:
ind160DS3=np.where(m33phot_test['DETSIG160']<3.0)
m33phot_test.remove_rows(ind160DS3)
m33phot_test

ID,X,Y,PIXRAD,FX,FY,RA,DEC,NUMOBJ,VIEWFRAC,CFRAC,OFRAC,GFRAC,EFRAC,CCNT,CPCNT,GCNT,GPCNT,ECNT,EPCNT,CLASSIFICATION_COUNT,PRIMARY_APCAN,SUBIMG,BRICK,FIELD,IMG,TOOTH,SUBJECT_ID,NUMOBJALL,NOSOLFLAG,EDGES,VIEWFRACS,EXCLUDEFLAGS,PIXHLR475,MAG336,SIG336,DETSIG336,MAG475,SIG475,DETSIG475,MAG814,SIG814,DETSIG814,MAG110,SIG110,DETSIG110,MAG160,SIG160,DETSIG160
int32,float32,float32,float32,float32,float32,float64,float64,int32,float32,float32,float32,float32,float32,int32,int32,int32,int32,int32,int32,int16,int32,bytes17,int16,int16,int16,int16,int32,int32,int16,bytes36,bytes49,bytes14,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
4,274.12375,430.10718,38.054737,1524.1238,430.10718,23.43433303773043,30.51408232617703,1,1.0,0.98333335,0.016666668,0.0,0.016666668,59,0,0,0,1,0,60,21254,M33-B03-F16_03,3,16,3,-1,28525713,2,0,"69.9,28.9,","1.0000,0.9000,","0,1,",13.384977,17.207281,0.07225124,9.632259,19.698397,0.08457241,12.864533,18.16659,0.15653984,6.9838634,17.306377,0.16535696,6.616979,16.484745,0.15090834,7.2411957
5,224.73451,423.41208,37.18926,849.7345,1623.4121,23.583845918691235,30.659212182808222,1,0.9836066,0.9166667,0.083333336,0.0,0.083333336,55,0,0,0,5,0,61,9815,M33-B02-F07_14,2,7,14,-1,28524968,2,0,"76.6,23.6,","0.9836,0.8548,","0,1,",9.4648075,18.9727,0.11130678,6.2656794,18.70199,0.023569176,46.085743,17.94467,0.06835622,15.904741,17.577536,0.12523706,8.708436,17.123743,0.15666679,6.978568
7,248.0658,140.99915,39.348427,873.0658,140.99915,23.390373394289195,30.6674512161078,2,0.96694213,0.974359,0.025641026,0.0,0.025641026,114,0,0,0,3,0,121,11623,M33-B02-F12_02,2,12,2,-1,28525096,2,0,"83.6,141.0,","0.9833,0.9508,","0,0,",12.779946,19.286543,0.14845498,4.710332,18.826597,0.08265972,13.160799,17.740986,0.16589268,6.5956807,17.166212,0.20642294,5.3231254,16.474852,0.23456143,4.7006655
8,203.50774,440.90045,63.505905,1453.5077,2440.9004,23.479685106627514,30.799434542708635,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,4403,M33-B01-F11_23,1,11,23,-1,28524585,2,1,"41.7,59.1,","0.9833,0.9833,","0,1,",16.818165,16.588139,0.054756697,12.701803,16.79031,0.03153818,34.436665,16.108368,0.0520216,20.886917,15.6878,0.10127982,10.751449,15.209545,0.14551604,7.505982
13,201.16205,254.97142,49.47727,201.16205,1854.9714,23.56453518259861,30.70314723396164,1,0.98333335,0.9661017,0.033898305,0.0,0.033898305,57,0,0,0,2,0,60,7830,M33-B02-F02_17,2,2,17,-1,28524831,2,0,"201.2,22.6,","0.9833,0.9333,","0,1,",15.648331,17.925293,0.050608266,13.744537,18.142805,0.0857727,12.6848545,17.062899,0.08403791,12.945444,16.555428,0.10290856,10.582573,15.656208,0.08346023,13.035229
17,279.86597,185.6878,50.159744,1529.866,2585.6877,23.52419243700545,30.83246332535244,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,1496,M33-B01-F04_27,1,4,27,-1,28524393,1,0,185.7,0.9833,0,12.008762,17.310535,0.0974104,7.152913,17.800167,0.0681135,15.961256,17.484009,0.08811067,12.349536,17.155066,0.13650873,7.995891,16.804056,0.15862875,6.8934984
20,276.76883,297.69016,30.20908,276.76883,297.69016,23.598355830088337,30.713668563851773,1,0.98333335,0.9830508,0.016949153,0.0,0.016949153,58,0,0,0,1,0,60,5263,M33-B01-F14_01,1,14,1,-1,28524647,1,0,202.3,0.9833,0,8.323929,19.603678,0.040604774,17.152548,19.532879,0.050141405,21.671915,19.035606,0.15924154,6.867064,18.891033,0.18725759,5.856528,18.556032,0.34349442,3.265872
21,578.23376,123.59756,41.767193,1203.2338,123.59756,23.408321861774905,30.55145010099244,2,0.96666664,0.98275864,0.01724138,0.00862069,0.00862069,114,0,1,0,1,0,120,19182,M33-B03-F11_02,3,11,2,-1,28525572,2,0,"123.6,100.1,","0.9833,0.9500,","0,0,",8.692522,17.169436,0.0254752,27.29495,17.810728,0.047361944,22.939564,17.308846,0.08559723,12.71059,16.69304,0.109354645,9.962718,16.129318,0.19476992,5.6342287
22,142.84187,198.62428,40.89402,142.84187,2198.6243,23.52113346551495,30.67682214846574,2,0.9590164,0.974359,0.025641026,0.0,0.025641026,114,

In [13]:
m33goodphot_5filts_no275=m33phot_test
m33goodphot_5filts_no275.write('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M33-Triangulum/Photometry_Tables/m33goodphot_5filts_no275.fits',
                 format='fits')
